In [74]:
# imports
import os
import pathlib
import pandas as pd
import cobra

In [ ]:
# get data directory
current_dir = pathlib.Path(os.getcwd())
data_dir = current_dir.parent.parent / "datasets"

c:\Users\phili\OneDrive - uni-bielefeld.de\Uni\SoSe24\BA\thesis_scripts\sMOMENT
c:\Users\phili\OneDrive - uni-bielefeld.de\Uni\SoSe24\BA\datasets


In [ ]:
# Read the first sheet of the Excel file into a DataFrame
all_prot = pd.read_excel(data_dir / "A735_protein_data_CCM_PRM_raw_data.xlsx")

In [70]:
print(f"Number of measured Proteins: {all_prot["Protein Accession"].nunique()}")
all_prot.head()

Number of measured Proteins: 81


,Protein Accession,Protein,Peptide Sequence,Precursor Mz,Product Mz,Isotope Label Type,Total Area,BioReplicate,Condition
0,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,1991927.0,1.0,SQ
1,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,2491982.0,2.0,SQ
2,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,2275862.0,3.0,SQ
3,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,2408610.0,1.0,Liver Mets.
4,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,2042891.0,2.0,Liver Mets.


In [52]:
primary_prot = all_prot.loc[all_prot["Condition"] == "SQ"]

primary_prot.head(5)

,Protein Accession,Protein,Peptide Sequence,Precursor Mz,Product Mz,Isotope Label Type,Total Area,BioReplicate,Condition
0,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,1991927.0,1.0,SQ
1,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,2491982.0,2.0,SQ
2,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,2275862.0,3.0,SQ
15,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,792.425030,light,1991927.0,1.0,SQ
16,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,792.425030,light,2491982.0,2.0,SQ


In [53]:
primary_prot.loc[primary_prot["Isotope Label Type"] == "heavy", ["Total Area"]]

,Total Area
45,1.223529e+09
46,9.456593e+08
47,1.091832e+09
60,1.223529e+09
61,9.456593e+08
...,...
14229,2.441980e+08
14230,5.585936e+08
14243,2.855180e+08
14244,2.441980e+08


In [55]:
primary_prot.rename(columns={"Total Area": "Heavy Spike Area"}).loc[primary_prot["Isotope Label Type"] == "heavy", ["Heavy Spike Area"]].reset_index(drop=True)

,Heavy Spike Area
0,1.223529e+09
1,9.456593e+08
2,1.091832e+09
3,1.223529e+09
4,9.456593e+08
...,...
1417,2.441980e+08
1418,5.585936e+08
1419,2.855180e+08
1420,2.441980e+08


In [56]:
primary_comparison = pd.concat(
    [
        primary_prot.loc[primary_prot["Isotope Label Type"] == "light", ["Protein Accession", "Protein", "Peptide Sequence", "BioReplicate", "Total Area"]].reset_index(drop=True), 
        primary_prot.rename(columns={"Total Area": "Heavy Spike Area"}).loc[primary_prot["Isotope Label Type"] == "heavy", ["Heavy Spike Area"]].reset_index(drop=True)
    ], 
    axis=1
    )
primary_comparison.head()

,Protein Accession,Protein,Peptide Sequence,BioReplicate,Total Area,Heavy Spike Area
0,P52209,6PGD_HUMAN,VDDFLANEAK,1.0,1991927.0,1.223529e+09
1,P52209,6PGD_HUMAN,VDDFLANEAK,2.0,2491982.0,9.456593e+08
2,P52209,6PGD_HUMAN,VDDFLANEAK,3.0,2275862.0,1.091832e+09
3,P52209,6PGD_HUMAN,VDDFLANEAK,1.0,1991927.0,1.223529e+09
4,P52209,6PGD_HUMAN,VDDFLANEAK,2.0,2491982.0,9.456593e+08


In [58]:
primary_comparison["Relative Protein Content"] = primary_comparison["Total Area"] / primary_comparison["Heavy Spike Area"]
primary_comparison.head()

,Protein Accession,Protein,Peptide Sequence,BioReplicate,Total Area,Heavy Spike Area,Relative Protein Content
0,P52209,6PGD_HUMAN,VDDFLANEAK,1.0,1991927.0,1.223529e+09,0.001628
1,P52209,6PGD_HUMAN,VDDFLANEAK,2.0,2491982.0,9.456593e+08,0.002635
2,P52209,6PGD_HUMAN,VDDFLANEAK,3.0,2275862.0,1.091832e+09,0.002084
3,P52209,6PGD_HUMAN,VDDFLANEAK,1.0,1991927.0,1.223529e+09,0.001628
4,P52209,6PGD_HUMAN,VDDFLANEAK,2.0,2491982.0,9.456593e+08,0.002635


In [ ]:
# sample weight: 10 ng = 10 * 10e-9 g =  0.00000001 g
# amount of heavy peptide: 50 fmol = 50 * 10e-12 mmol = 0.00000000005 mmol 
primary_comparison["Absolute Protein Content [mmol/gDW]"] = primary_comparison["Relative Protein Content"] * 0.00000000005 / 0.00000001
primary_comparison.head()


,Protein Accession,Protein,Peptide Sequence,BioReplicate,Total Area,Heavy Spike Area,Relative Protein Content,Absolute Protein Content [mmol/gDW]
0,P52209,6PGD_HUMAN,VDDFLANEAK,1.0,1991927.0,1.223529e+09,0.001628,0.000008
1,P52209,6PGD_HUMAN,VDDFLANEAK,2.0,2491982.0,9.456593e+08,0.002635,0.000013
2,P52209,6PGD_HUMAN,VDDFLANEAK,3.0,2275862.0,1.091832e+09,0.002084,0.000010
3,P52209,6PGD_HUMAN,VDDFLANEAK,1.0,1991927.0,1.223529e+09,0.001628,0.000008
4,P52209,6PGD_HUMAN,VDDFLANEAK,2.0,2491982.0,9.456593e+08,0.002635,0.000013


In [66]:
primary_comparison["Protein Accession"].nunique()

81

In [73]:
primary_conc = primary_comparison.groupby(["Protein Accession"])["Absolute Protein Content [mmol/gDW]"].agg(['mean', 'std']).reset_index()
primary_conc.head()

,Protein Accession,mean,std
0,O00330,0.000009,5.308544e-06
1,O00757,0.000004,2.432937e-06
2,O14556,0.000002,2.998146e-07
3,O43837,0.000008,1.889814e-06
4,O75390,0.000022,2.217726e-05


Calculating the 

In [ ]:
# extracting uniprot accession numbers from ihuman for easy access
ihuman = cobra.io.read_sbml_model(current_dir.parent / "models" / "Human-GEM.xml")

In [ ]:
# retrieving protein accession numbers to select protein data
accession_numbers = []
for gene in ihuman.genes:
    accession_numbers.append(gene.annotation["uniprot"])

print(accession_numbers)

['O60762', 'Q9BTY2', 'P48506', 'Q16850', 'P28838', 'O14792', 'P19801', 'Q76N89', 'Q9NR63', 'Q9Y216', 'P52569', 'P54819', 'P28907', 'Q02790', 'O14561', 'Q16654', 'Q86VW1', 'Q9UJS0', 'P02730', 'P53701', 'P05141', 'P52435', 'Q53FZ2', 'Q92793', 'P05164', 'P27169', 'Q9UKG9', 'P21439', 'O75592', 'Q15119', 'Q9NZC3', 'Q09428', 'Q53H12', 'P43353', 'O75223', 'Q12887', 'P51689', 'P41247', 'O95045', 'P43034', 'P35228', 'Q13183', 'P35499', 'P51854', 'Q9BRB3', 'P31513', 'Q8WY64', 'O95544', 'Q9UDR5', 'P10620', 'O75594', 'Q11201', 'Q15386', 'O60673', 'Q9UKY4', 'P31930', 'Q9NSD5', 'P22304', 'Q01740', 'O95372', 'Q99884', 'Q8WTS1', 'Q9NWF9', 'O00443', 'O43909', 'Q9NYP7', 'P09917', 'Q8N806', 'Q9Y2E5', 'Q9BZJ4', 'O95394', 'Q9UIY3', 'Q9NW08', 'Q9UBS8', 'Q14181', 'P15309', 'P40925', 'P16444', 'Q9UHC9', 'Q9H1B5', 'Q8NE62', 'Q68CQ7', 'P98194', 'Q8WUX1', 'Q15311', 'P49279', 'O00116', 'P50993', 'Q07973', 'Q9HB55', 'P82251', 'P31327', 'Q9H6R6', 'P28331', 'P13196', 'Q9NS18', 'Q9Y315', 'Q92887', 'P48507', 'Q8IWV8',

In [90]:
# retrieving gene symbols to calculate relative amount of total protein that is present in the model
gene_symbols = []
for gene in ihuman.genes:
    gene_symbols.append(gene.annotation["hgnc.symbol"])

print(gene_symbols)

['DPM1', 'FUCA2', 'GCLC', 'CYP51A1', 'LAP3', 'HS3ST1', 'AOC1', 'HECW1', 'CYP26B1', 'MTMR7', 'SLC7A2', 'AK2', 'CD38', 'FKBP4', 'NDUFAB1', 'PDK4', 'SLC22A16', 'SLC25A13', 'SLC4A1', 'HCCS', 'SLC25A5', 'POLR2J', 'ACSM3', 'CREBBP', 'MPO', 'PON1', 'CROT', 'ABCB4', 'MYCBP2', 'PDK2', 'GDE1', 'ABCC8', 'AGK', 'ALDH3B1', 'GGCT', 'COX10', 'ARSD', 'PNPLA4', 'UPP2', 'PAFAH1B1', 'NOS2', 'SLC13A2', 'SCN4A', 'TKTL1', 'PIGQ', 'FMO3', 'MYLIP', 'NADK', 'AASS', 'MGST1', 'PGLYRP1', 'ST3GAL1', 'UBE3C', 'REV3L', 'POMT2', 'UQCRC1', 'SLC6A13', 'IDS', 'FMO1', 'LYPLA2', 'SLC6A7', 'ABHD5', 'RNF216', 'PIK3C2A', 'EXTL3', 'ELOVL5', 'ALOX5', 'UBR7', 'MAN2B2', 'SLC25A39', 'PGM3', 'RWDD2A', 'POLR3B', 'RNF14', 'POLA2', 'ACP3', 'MDH1', 'DPEP1', 'NPC1L1', 'XYLT2', 'CHDH', 'GLT8D1', 'ATP2C1', 'SLC38A5', 'RALBP1', 'SLC11A1', 'AGPS', 'ATP1A2', 'CYP24A1', 'CYP3A43', 'SLC7A9', 'CPS1', 'ZDHHC6', 'NDUFS1', 'ALAS1', 'GLRX2', 'DERA', 'ABCC2', 'GCLM', 'UBR2', 'HSD17B6', 'TYMP', 'KPNA6', 'AGPAT4', 'B4GALT7', 'NUP160', 'SARS1', 'PNPLA

In [83]:
# calculating amount of metabolic protein from Cell model passport proteomics data
# https://cellmodelpassports.sanger.ac.uk/downloads - proteomics_all_20221214.csv (accessed 15/12/2024)

protomics_data = pd.read_csv(data_dir / "cell_model-passport_proteomics_all_20221214" / "proteomics_all_20221214.csv")
protomics_data.head()

,uniprot_id,model_id,model_name,protein_intensity,zscore,symbol
0,A0A075B6K4,SIDM00483,SK-GT-4,-0.577590,-1.284310,IGLV3-10
1,A0A075B6K4,SIDM00689,JM1,3.408550,0.129406,IGLV3-10
2,A0A075B6K4,SIDM01259,GR-ST,6.300060,1.154900,IGLV3-10
3,A0A075B6N1,SIDM00846,HeLa,0.127721,-1.034210,TRBV19
4,A0A075B6N1,SIDM00958,CML-T1,4.283890,1.352450,TRBV19
